In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import random
from tabulate import tabulate
from time import perf_counter

In [2]:
#Paso 1

def get_datos():
    datos=[[  2.01666708 ,  56.18031474], [  4.79734083 ,  47.18848199], [  9.23784581 ,  57.68974048], [ 14.11529384 ,  43.70348368],
         [ 14.92688637 ,  59.10244323], [ 17.34408196 ,  65.96080804], [ 17.62435324 ,  45.74334603], [ 22.41875021 ,  13.575581  ],
         [ 25.3139145  ,  68.43756969], [ 34.85886672 , 147.15375307], [ 38.87476262 ,  25.39687009], [ 42.01380169 ,  74.39010069],
         [ 46.63551059 ,  98.93395801], [ 49.58578273 , 116.07013679], [ 50.18371003 , 138.55546747], [ 52.06630172 , 139.36601894],
         [ 54.68810274 , 150.09622546], [ 57.13046193 , 156.14375739], [ 66.35609935 , 119.75844452], [ 69.05499042 , 139.08155228],
         [ 69.51252436 , 128.72247348], [ 69.83788756 , 152.65110462], [ 79.76649746 , 148.23106977], [ 81.83730552 , 137.86314926],
         [ 87.09879038 , 217.28932067], [ 89.00469759 , 168.64994509], [ 93.17139213 , 163.10598352], [ 93.66070686 , 200.47638924],
         [ 94.1944751  , 150.44019156], [ 97.36920633 , 173.2055957 ]]

    return datos


datos=np.array(get_datos())

x=datos[:,0]
y=datos[:,1]


In [3]:
#Paso 2
def compile_fit(capas,x,y):
    np.random.seed(5)
    tf.random.set_seed(5)
    random.seed(5)  
    
    model=Sequential()
    for index,neuronas_capa in enumerate(capas):
        if (index==0):
            model.add(Dense(neuronas_capa, activation='relu',input_dim=1))
        elif (index==len(capas)-1):
            model.add(Dense(neuronas_capa, activation='linear'))
        else:
            model.add(Dense(neuronas_capa, activation='relu'))
    model.compile(loss='mean_squared_error')
    
    history=model.fit(x, y,epochs=5000,verbose=False) 

    return model,history

capas=[5,10,3,10,1]
model,history=compile_fit(capas,x,y)



2024-11-23 20:44:15.831203: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-11-23 20:44:15.831227: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: beren
2024-11-23 20:44:15.831232: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: beren
2024-11-23 20:44:15.831334: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 565.57.1
2024-11-23 20:44:15.831353: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 565.57.1
2024-11-23 20:44:15.831358: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 565.57.1
2024-11-23 20:44:15.831815: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operatio

In [4]:
#Paso 3

x_paso3=np.array([ 2.01666708, 4.79734083, 9.23784581, 14.11529384 ]).reshape(-1,1)
y_true_paso3=np.array([ 56.18031474, 47.18848199, 57.68974048, 43.70348368]).reshape(-1,1)
y_pred_paso3=model.predict(x_paso3)
y_error_paso3=(y_true_paso3-y_pred_paso3)**2

resultado_paso3=np.column_stack((x_paso3,y_true_paso3,y_pred_paso3,y_error_paso3))

print(tabulate(resultado_paso3,headers=["x","y_true","y_pred","error"]))


       x    y_true    y_pred     error
--------  --------  --------  --------
 2.01667   56.1803   36.5675  384.663
 4.79734   47.1885   40.9242   39.2418
 9.23785   57.6897   47.8814   96.2036
14.1153    43.7035   55.5232  139.707


In [5]:
#Paso 4

mse=np.sum(y_error_paso3)/np.size(y_error_paso3)
print(f"Mean Squared Error (MSE)={mse:.3f}")

Mean Squared Error (MSE)=164.954


In [6]:
#Paso 5
media_valores_verdaderos=np.sum(y_true_paso3)/np.size(y_true_paso3)
suma_errores=np.sum((y_true_paso3-y_pred_paso3)**2)
suma_valores_media=np.sum((y_true_paso3-media_valores_verdaderos)**2)
coeficiente_determinacion=1-(suma_errores/suma_valores_media)

print(f"Coeficiente de determinación o R²={coeficiente_determinacion:.3f}")

Coeficiente de determinación o R²=-3.740


In [7]:
#Paso 6
def get_mse(y_true,y_pred):
    y_error=(y_true-y_pred)**2
    mse=np.sum(y_error)/np.size(y_error)

    return mse

def get_coeficiente_determinacion(y_true,y_pred):
    media_valores_verdaderos=np.sum(y_true)/np.size(y_true)
    suma_errores=np.sum((y_true-y_pred)**2)
    suma_valores_media=np.sum((y_true-media_valores_verdaderos)**2)
    coeficiente_determinacion=1-(suma_errores/suma_valores_media)

    return coeficiente_determinacion


def get_metricas_modelo(model,x,y_true):
    y_pred=model.predict(x)
    mse=get_mse(y_true,y_pred)
    coeficiente_determinacion=get_coeficiente_determinacion(y_true,y_pred)

    return mse,coeficiente_determinacion

mse,coeficiente_determinacion=get_metricas_modelo(model,x.reshape(-1,1),y.reshape(-1,1))

print(f"Coeficiente de determinación o R²={coeficiente_determinacion:.3f}")
print(f"Mean Squared Error (MSE)={mse:.3f}")

Coeficiente de determinación o R²=0.748
Mean Squared Error (MSE)=718.298


In [8]:
#Paso 7
capas=[20,40,80,40,20,1]
model,history=compile_fit(capas,x,y)
mse,coeficiente_determinacion=get_metricas_modelo(model,x.reshape(-1,1),y.reshape(-1,1))

print(f"Coeficiente de determinación o R²={coeficiente_determinacion:.3f}")
print(f"Mean Squared Error (MSE)={mse:.3f}")

Coeficiente de determinación o R²=0.811
Mean Squared Error (MSE)=539.139


In [9]:
#Paso 8

def compile_fit(capas,x,y,activation):
    np.random.seed(5)
    tf.random.set_seed(5)
    random.seed(5)  
    
    model=Sequential()
    for index,neuronas_capa in enumerate(capas):
        if (index==0):
            model.add(Dense(neuronas_capa, activation=activation,input_dim=1))
        elif (index==len(capas)-1):
            model.add(Dense(neuronas_capa, activation='linear'))
        else:
            model.add(Dense(neuronas_capa, activation=activation))
    model.compile(loss='mean_squared_error')
    
    history=model.fit(x, y,epochs=5000,verbose=False) 

    return model,history



capas=[20,40,80,40,20,1]
model,history=compile_fit(capas,x,y,"selu")
mse,coeficiente_determinacion=get_metricas_modelo(model,x.reshape(-1,1),y.reshape(-1,1))

print(f"Coeficiente de determinación o R²={coeficiente_determinacion:.3f}")
print(f"Mean Squared Error (MSE)={mse:.3f}")

Coeficiente de determinación o R²=0.936
Mean Squared Error (MSE)=182.719


In [10]:
#Paso 9

def compile_fit(capas,x,y,activation):
    np.random.seed(5)
    tf.random.set_seed(5)
    random.seed(5)  
    
    model=Sequential()
    for index,neuronas_capa in enumerate(capas):
        if (index==0):
            model.add(Dense(neuronas_capa, activation=activation,input_dim=1))
        elif (index==len(capas)-1):
            model.add(Dense(neuronas_capa, activation='linear'))
        else:
            model.add(Dense(neuronas_capa, activation=activation))
    model.compile(loss='mean_squared_error')
    
    tiempo_entrenamiento = perf_counter()
    history=model.fit(x, y,epochs=5000,verbose=False) 
    tiempo_entrenamiento=perf_counter()-tiempo_entrenamiento

    return model,history,tiempo_entrenamiento

redes_neuronales=[
    [[5,10,3,10,1],"relu"],
    [[5,10,3,10,1],"selu"],
    [[5,10,3,10,1],"tanh"],    
    [[20,40,80,40,20,1],"relu"],
    [[20,40,80,40,20,1],"selu"],
    [[20,40,80,40,20,1],"tanh"],
    [[20,40,80,160,80,40,20,1],"relu"],
    [[20,40,80,160,80,40,20,1],"selu"],
    [[20,40,80,160,80,40,20,1],"tanh"]                           
]

resultados=[]

for index,(capas,activacion) in enumerate(redes_neuronales):

    model,history,tiempo_entrenamiento=compile_fit(capas,x,y,activacion)
    mse,coeficiente_determinacion=get_metricas_modelo(model,x.reshape(-1,1),y.reshape(-1,1))

    resultados.append([str(index+1),str(capas),activacion,mse,coeficiente_determinacion,tiempo_entrenamiento])

print(tabulate(resultados,headers=["Nombre\nRed","Capas","Activación","MSE","R²","Tiempo\nEntrenamiento"]))


  Nombre  Capas                             Activación         MSE           R²           Tiempo
     Red                                                                           Entrenamiento
--------  --------------------------------  ------------  --------  -----------  ---------------
       1  [5, 10, 3, 10, 1]                 relu           718.298   0.748316            3.4089
       2  [5, 10, 3, 10, 1]                 selu           717.653   0.748541            3.42228
       3  [5, 10, 3, 10, 1]                 tanh          5941.53   -1.08185             3.40019
       4  [20, 40, 80, 40, 20, 1]           relu           539.139   0.811091            3.6182
       5  [20, 40, 80, 40, 20, 1]           selu           182.719   0.935977            3.65107
       6  [20, 40, 80, 40, 20, 1]           tanh          2883.8    -0.0104537           3.8594
       7  [20, 40, 80, 160, 80, 40, 20, 1]  relu           396.495   0.861072            4.21018
       8  [20, 40, 80, 160, 80, 4

In [11]:
def get_datos_validacion():
    datos_validacion=[[  1.22140488 , 59.35315077] , [  2.42834632 ,  3.50613409] , [  4.27529991 , 70.39938914] ,
        [ 14.44651349 , 50.0606769 ] , [ 16.10795855 , 81.08562061] , [ 16.75024181 , 33.95365822] ,
        [ 26.80487149 , 47.1495392 ] , [ 28.81517859 ,106.34919698] , [ 48.56698654 ,120.25398606] ,
        [ 52.08015067 ,116.7993955 ] , [ 53.30646055 ,131.30936472] , [ 55.09968806 ,131.34281752] ,
        [ 60.39615207 , 97.77483743] , [ 73.52487026 , 92.30645543] , [ 76.2771471  ,109.9995226 ] ,
        [ 84.56808303 ,120.60657657] , [ 89.2700557  ,117.3687155 ] , [ 91.03720679 ,159.47376137] ,
        [ 93.53406333 ,166.44439331] , [ 94.78103495 ,180.66942656]]
    return datos_validacion

datos_validacion=np.array(get_datos_validacion())
x_validacion=datos_validacion[:,0]
y_validacion=datos_validacion[:,1]


resultados=[]

for index,(capas,activacion) in enumerate(redes_neuronales):

    model,history,tiempo_entrenamiento=compile_fit(capas,x,y,activacion)
    mse_entrenamiento,coeficiente_determinacion_entrenamiento=get_metricas_modelo(model,x.reshape(-1,1),y.reshape(-1,1))
    mse_validacion,coeficiente_determinacion_validacion=get_metricas_modelo(model,x_validacion.reshape(-1,1),y_validacion.reshape(-1,1))   

    resultados.append([str(index+1),str(capas),activacion,mse_entrenamiento,coeficiente_determinacion_entrenamiento,tiempo_entrenamiento,mse_validacion,coeficiente_determinacion_validacion])

print(tabulate(resultados,headers=["Nombre\nRed","Capas","Activación","MSE\nEntrenamiento","R²\nEntrenamiento","Tiempo\nEntrenamiento","MSE\nValidación","R²\nValidación"]))

  Nombre  Capas                             Activación                MSE               R²           Tiempo           MSE            R²
     Red                                                    Entrenamiento    Entrenamiento    Entrenamiento    Validación    Validación
--------  --------------------------------  ------------  ---------------  ---------------  ---------------  ------------  ------------
       1  [5, 10, 3, 10, 1]                 relu                  718.298       0.748316            3.40629       862.985     0.567329
       2  [5, 10, 3, 10, 1]                 selu                  717.653       0.748541            3.43374       798.624     0.599598
       3  [5, 10, 3, 10, 1]                 tanh                 5941.53       -1.08185             3.46328      3753.3      -0.881774
       4  [20, 40, 80, 40, 20, 1]           relu                  539.139       0.811091            3.83408       846.146     0.575772
       5  [20, 40, 80, 40, 20, 1]           selu    

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import random
from tabulate import tabulate
from time import perf_counter

def get_datos():
    datos=[[  2.01666708 ,  56.18031474], [  4.79734083 ,  47.18848199], [  9.23784581 ,  57.68974048], [ 14.11529384 ,  43.70348368],
         [ 14.92688637 ,  59.10244323], [ 17.34408196 ,  65.96080804], [ 17.62435324 ,  45.74334603], [ 22.41875021 ,  13.575581  ],
         [ 25.3139145  ,  68.43756969], [ 34.85886672 , 147.15375307], [ 38.87476262 ,  25.39687009], [ 42.01380169 ,  74.39010069],
         [ 46.63551059 ,  98.93395801], [ 49.58578273 , 116.07013679], [ 50.18371003 , 138.55546747], [ 52.06630172 , 139.36601894],
         [ 54.68810274 , 150.09622546], [ 57.13046193 , 156.14375739], [ 66.35609935 , 119.75844452], [ 69.05499042 , 139.08155228],
         [ 69.51252436 , 128.72247348], [ 69.83788756 , 152.65110462], [ 79.76649746 , 148.23106977], [ 81.83730552 , 137.86314926],
         [ 87.09879038 , 217.28932067], [ 89.00469759 , 168.64994509], [ 93.17139213 , 163.10598352], [ 93.66070686 , 200.47638924],
         [ 94.1944751  , 150.44019156], [ 97.36920633 , 173.2055957 ]]

    return datos

def get_datos_validacion():
    datos_validacion=[[  1.22140488 , 59.35315077] , [  2.42834632 ,  3.50613409] , [  4.27529991 , 70.39938914] ,
        [ 14.44651349 , 50.0606769 ] , [ 16.10795855 , 81.08562061] , [ 16.75024181 , 33.95365822] ,
        [ 26.80487149 , 47.1495392 ] , [ 28.81517859 ,106.34919698] , [ 48.56698654 ,120.25398606] ,
        [ 52.08015067 ,116.7993955 ] , [ 53.30646055 ,131.30936472] , [ 55.09968806 ,131.34281752] ,
        [ 60.39615207 , 97.77483743] , [ 73.52487026 , 92.30645543] , [ 76.2771471  ,109.9995226 ] ,
        [ 84.56808303 ,120.60657657] , [ 89.2700557  ,117.3687155 ] , [ 91.03720679 ,159.47376137] ,
        [ 93.53406333 ,166.44439331] , [ 94.78103495 ,180.66942656]]
    return datos_validacion

def compile_fit(capas,x,y,activation):
    np.random.seed(5)
    tf.random.set_seed(5)
    random.seed(5)  
    
    model=Sequential()
    for index,neuronas_capa in enumerate(capas):
        if (index==0):
            model.add(Dense(neuronas_capa, activation=activation,input_dim=1))
        elif (index==len(capas)-1):
            model.add(Dense(neuronas_capa, activation='linear'))
        else:
            model.add(Dense(neuronas_capa, activation=activation))
    model.compile(loss='mean_squared_error')
    
    tiempo_entrenamiento = perf_counter()
    history=model.fit(x, y,epochs=5000,verbose=False) 
    tiempo_entrenamiento=perf_counter()-tiempo_entrenamiento

    return model,history,tiempo_entrenamiento

def get_mse(y_true,y_pred):
    y_error=(y_true-y_pred)**2
    mse=np.sum(y_error)/np.size(y_error)

    return mse

def get_coeficiente_determinacion(y_true,y_pred):
    media_valores_verdaderos=np.sum(y_true)/np.size(y_true)
    suma_errores=np.sum((y_true-y_pred)**2)
    suma_valores_media=np.sum((y_true-media_valores_verdaderos)**2)
    coeficiente_determinacion=1-(suma_errores/suma_valores_media)

    return coeficiente_determinacion


def get_metricas_modelo(model,x,y_true):
    y_pred=model.predict(x)
    mse=get_mse(y_true,y_pred)
    coeficiente_determinacion=get_coeficiente_determinacion(y_true,y_pred)

    return mse,coeficiente_determinacion


datos=np.array(get_datos())
x=datos[:,0]
y=datos[:,1]

datos_validacion=np.array(get_datos_validacion())
x_validacion=datos_validacion[:,0]
y_validacion=datos_validacion[:,1]


redes_neuronales=[
    [[5,10,3,10,1],"relu"],
    [[5,10,3,10,1],"selu"],
    [[5,10,3,10,1],"tanh"],    
    [[20,40,80,40,20,1],"relu"],
    [[20,40,80,40,20,1],"selu"],
    [[20,40,80,40,20,1],"tanh"],
    [[20,40,80,160,80,40,20,1],"relu"],
    [[20,40,80,160,80,40,20,1],"selu"],
    [[20,40,80,160,80,40,20,1],"tanh"]                           
]


resultados=[]

for index,(capas,activacion) in enumerate(redes_neuronales):

    model,history,tiempo_entrenamiento=compile_fit(capas,x,y,activacion)
    mse_entrenamiento,coeficiente_determinacion_entrenamiento=get_metricas_modelo(model,x.reshape(-1,1),y.reshape(-1,1))
    mse_validacion,coeficiente_determinacion_validacion=get_metricas_modelo(model,x_validacion.reshape(-1,1),y_validacion.reshape(-1,1))   

    resultados.append([str(index+1),str(capas),activacion,mse_entrenamiento,coeficiente_determinacion_entrenamiento,tiempo_entrenamiento,mse_validacion,coeficiente_determinacion_validacion])

print(tabulate(resultados,headers=["Nombre\nRed","Capas","Activación","MSE\nEntrenamiento","R²\nEntrenamiento","Tiempo\nEntrenamiento","MSE\nValidación","R²\nValidación"]))

2024-11-23 20:49:11.365554: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-11-23 20:49:11.365577: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: beren
2024-11-23 20:49:11.365583: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: beren
2024-11-23 20:49:11.365652: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 565.57.1
2024-11-23 20:49:11.365670: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 565.57.1
2024-11-23 20:49:11.365674: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 565.57.1
2024-11-23 20:49:11.365840: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operatio

  Nombre  Capas                             Activación                MSE               R²           Tiempo           MSE            R²
     Red                                                    Entrenamiento    Entrenamiento    Entrenamiento    Validación    Validación
--------  --------------------------------  ------------  ---------------  ---------------  ---------------  ------------  ------------
       1  [5, 10, 3, 10, 1]                 relu                  718.298       0.748316            3.60173       862.985     0.567329
       2  [5, 10, 3, 10, 1]                 selu                  717.653       0.748541            3.52216       798.624     0.599598
       3  [5, 10, 3, 10, 1]                 tanh                 5941.53       -1.08185             3.51282      3753.3      -0.881774
       4  [20, 40, 80, 40, 20, 1]           relu                  539.139       0.811091            3.64414       846.146     0.575772
       5  [20, 40, 80, 40, 20, 1]           selu    